### Example to mimic old PTW

This is a notebook with parameter values set to mimic the old PTW calibration

In [1]:
using Pkg
pkg"activate ."
pkg"instantiate"
pkg"precompile"

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
Precompiling project...


In [2]:
using PerlaTonettiWaugh, Plots, BenchmarkTools

┌ Info: Recompiling stale cache file /Users/arnavsood/.julia/compiled/v1.1/PerlaTonettiWaugh/Zi3ee.ji for PerlaTonettiWaugh [d2cacc76-b06b-11e8-362b-f18d03a16405]
└ @ Base loading.jl:1184


In [3]:
using Interpolations, Sundials, Parameters

Below is the calibration, the gbm and death rate set to be small values. The otheres are virtually taken from Tabe 1. The only ones that are a bit fuzzy are $\kappa$ which was rounded up and then the $\chi$ which is a bit higher. I beleve that this may have been a typo in the paper.

In [4]:
parameters_old_paper = parameter_defaults(d = 4.0,
                                           θ = 3.1878,
                                           κ = 0.006,
                                           χ = 1.00/2.80,
                                           υ = 0.001,
                                           σ = 3.0,
                                           ζ = 1.00,
                                           δ = 0.001)

settings_old_paper_defaults = @with_kw (z_max = 5,
                                z_ex = unique([range(0., 0.1, length = 400)' range(0.1, 1., length = 400)' range(1., 10.0, length = 200+2)']),
                                z = z_ex[2:end-1],
                                Δ_E = 1e-6,
                                ode_solve_algorithm = CVODE_BDF(),
                                T = 120.0,
                                t = range(0.0, T, length = 10),
                                g = LinearInterpolation(t, stationary_numerical(parameter_defaults(), z).g .+ 0.01*t),
                                weights = [10.0; fill(1.0, 13)],
                                transition_x0 = [-0.9802869871313153, -0.7679611162133799, -0.6483822140201239, -0.5709998420691726, -0.4410497194161549, -0.35188633823047205, -0.28134090933192113, -0.22721306548238096, -0.2132657066634307, -0.1802989139615504, -0.1407983331567128, -0.10561616300315106, -0.08546763464126883, -0.058948603687082865, -0.02960294672034148, -0.020649289609280547, -0.013922070758445242, -0.008451708149201357, -0.0039236251615955165],
                                fifty_node_iv = [-1.00157, -0.848157, -0.821211, -0.821211, -0.821211, -0.748497, -0.633587, -0.527711, -0.498239, -0.498239, -0.498239, -0.498239, -0.3316, -0.3316, -0.3316, -0.3316, -0.3316, -0.281318, -0.281318, -0.281318, -0.281318, -0.281318, -0.241756, -0.230492, -0.168434, -0.168434, -0.168434, -0.168434, -0.105236, -0.103655, -0.103655, -0.0787871, -0.0787871, -0.0787871, -0.0787871, -0.0787871, -0.0787871, -0.0713765, -0.0713765, -0.0713765, -0.0713765, -0.0343871, -0.0334064, -0.0334064, -0.029373, -0.029373, -0.029373, -0.029373, -0.029373, -0.029373],
                                continuation_x0 = zeros(length(transition_x0)),
                                transition_lb = -ones(length(transition_x0)),
                                transition_ub = zeros(length(transition_x0)),
                                transition_iterations = 1000,
                                transition_penalty_coefficient = 0.0, # coefficient to be used for a penalty function for constraints on increasing E
                                T_U_bar = 50.0,
                                tstops = nothing)

#3 (generic function with 2 methods)

In [5]:
# Define common objects. 
parameters = parameters_old_paper
settings = settings_old_paper_defaults()
z_grid = settings.z
P = length(z_grid)

d_0 = 10.0 # This will get us near autarky.
d_T = parameters.d

params_0 = merge(parameters, (d = d_0, )) # parameters to be used at t = 0
params_T = merge(parameters, (d = d_T, )) # parameters to be used at t = T

# initial value for numerical solver on (g, z_hat, Omega)
initial_x = [0.02; 2; .57] # Play with this if it is kicking back errors

# solve for stationary solution at t = 0
stationary_sol_atuk = stationary_algebraic(params_0, initial_x) # solution at t = 0
stationary_sol = stationary_algebraic(params_T, initial_x) # solution at t = T

Ω_0 = stationary_sol_atuk.Ω;
Ω_T = stationary_sol.Ω;

MethodError: MethodError: no method matching L₁₋(::Array{Float64,1}, ::Tuple{SimpleDifferentialOperators.Mixed{Float64},SimpleDifferentialOperators.Mixed{Float64}})
Closest candidates are:
  L₁₋(::Any) at /Users/arnavsood/Research/SimpleDifferentialOperators.jl/src/operators.jl:279

In [6]:
changed_parameters = false # set this to true if you drastically change the parameters

false

In [7]:
# display_stationary_sol(stationary_sol)
# uncomment to see all parameters for a steady state

#### Comparison of Steady States

First notice that the growth rate is near similar in autarky to that in Table 2. Then the second part computes the consumption equivalent 

In [8]:
print(stationary_sol_atuk.g,"\n")

print(100*(stationary_sol.U_bar/stationary_sol_atuk.U_bar-1),"\n")

lambda_ss = 100*(consumption_equivalent(stationary_sol.U_bar, stationary_sol_atuk.U_bar, parameters)-1)

print("SS to SS welfare gain: ", lambda_ss,"\n")

UndefVarError: UndefVarError: stationary_sol_atuk not defined

So all this stuff is roughly matching up with what we had in Table 2. About a 1.38 growth rate, 13 percent increase in utility, 26 (24 in paper) in consumption units.

#### Transition Path

In [9]:
settings = merge(settings, (params_T = params_T, stationary_sol_T = stationary_numerical(params_T, z_grid), Ω_0 = Ω_0, transition_iterations = 0));

UndefVarError: UndefVarError: params_T not defined

In [10]:
if changed_parameters # will use the global solver to set a new initial condition, hopefully in the vicinity of the solution implied by the new parameters
    settings = merge(settings, (transition_iterations = 1000, ))
    @time result = solve_full_model_global(settings; impose_E_monotonicity_constraints = true)
    E_nodes = result.E_nodes;
    settings = merge(settings, (transition_x0 = E_nodes, ))
end

In [11]:
@time result = solve_full_model(settings; impose_E_monotonicity_constraints = true)
solved = result.solution;
E_nodes = result.E_nodes;
solved = solved.results;

UndefVarError: UndefVarError: settings not defined

In [12]:
plot_Ω = plot(solved.t, solved.Ω, label = "Omega", lw = 3)
plot_residual = plot(solved.t, solved.entry_residual, label = "entry_residual", lw = 3)
plot(plot_Ω, plot_residual, layout = (2,1))

UndefVarError: UndefVarError: solved not defined

In [13]:
plot1 = plot(solved.t, solved.g, label = "g", lw = 3)
plot2 = plot(solved.t, solved.z_hat, label = "z_hat", lw = 3)
plot3 = plot(solved.t, solved.S, label = "S", lw = 3)
plot4 = plot(solved.t, solved.entry_residual, label = "entry_residual", lw = 3)
plot(plot1, plot2, plot3, plot4, layout=(2,2), size = (800, 400))

UndefVarError: UndefVarError: solved not defined

#### Welfare Including Transition Path

Here it is

In [14]:
lambda_tpath = 100*(consumption_equivalent(solved.U[1], stationary_sol_atuk.U_bar, parameters)-1)

print("Inclusive of the Transition Path: ", lambda_tpath,"\n")

UndefVarError: UndefVarError: solved not defined

So the transition makes welfare INCREASE by a factor of 4....

### Saving results in .csv files

In [15]:
using CSV
CSV.write("old_pwt_tpath-results.csv", solved)

ArgumentError: ArgumentError: Package CSV not found in current path:
- Run `import Pkg; Pkg.add("CSV")` to install the CSV package.
